#GPT3.5による「和菓子コレクションweb生成」エージェント



In [ ]:
!pip -q install openai==0.27.8 tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.5 MB/s eta 0:00:00


ご自身のAPI keyを入れて下さい

In [ ]:
import openai
import os

openai.api_key = "#####"

## 1. web-site作成の関数を用意します

In [ ]:
from IPython.display import HTML

def web_site_builder(title: str, explanation: str):

    html = """<!DOCTYPE html>
<html>
    <head>
        <title>Product</title>
        <style>
            body {
                background-color: #f0f0f0;
                font-family: Arial, sans-serif;
            }
            .container {
                width: 80%;
                margin: auto;
                padding: 20px;
                background-color: #ffffff;
                box-shadow: 0px 0px 10px #bbbbbb;
                text-align: center;
            }
            h1 {
                color: #333333;
                margin-bottom: 20px;
            }
            p {
                color: #666666;
                font-size: 16px;
                line-height: 1.5;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>"""+title+"""</h1>
            <p>"""+explanation+"""</p>
        </div>
    </body>
</html>"""
    with open('index.html', 'w') as fp:
        fp.write(html)
    return HTML(filename='index.html')

## 2.function callingのためのプログラムを用意します

In [ ]:
web_site_builder_func = {
    "name": "web_site_builder",
    "description": "Creates product web-site",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {
                "type": "string",
                "description": "商品名。This should be written in Japanese."
            },
            "explanation": {
                "type": "string",
                "description": "美味しいお菓子の説明文。思わず買いたくなるようなかっこいい表現で説明。This should be written with 300 words in Japanese."
            }
        },
        "required": ["title", "explanation"]
    }
}

In [ ]:
type(web_site_builder_func)

dict

## 3.webのための商品名・説明文をGPT3.5-turboで作成します

In [ ]:
instruct = "Create a web-site for a pretty Japanese sweets collection"

result = openai.ChatCompletion.create(
    model='gpt-3.5-turbo-0613',
    messages=[{"role": "user", "content": instruct}],
    functions=[web_site_builder_func]
)

result

<OpenAIObject chat.completion id=chatcmpl-7SOZJOvntjm6OtXIIn3re7s7FsHaE at 0x7f224b7e1800> JSON: {
  "id": "chatcmpl-7SOZJOvntjm6OtXIIn3re7s7FsHaE",
  "object": "chat.completion",
  "created": 1687001981,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "web_site_builder",
          "arguments": "{\n  \"title\": \"\u30d7\u30ea\u30c6\u30a3\u30fc\u548c\u83d3\u5b50\u30b3\u30ec\u30af\u30b7\u30e7\u30f3\",\n  \"explanation\": \"\u30d7\u30ea\u30c6\u30a3\u30fc\u548c\u83d3\u5b50\u30b3\u30ec\u30af\u30b7\u30e7\u30f3\u306f\u3001\u65e5\u672c\u306e\u4f1d\u7d71\u7684\u306a\u548c\u83d3\u5b50\u3092\u96c6\u3081\u305f\u30b9\u30a4\u30fc\u30c4\u306e\u30b3\u30ec\u30af\u30b7\u30e7\u30f3\u3067\u3059\u3002\u7f8e\u3057\u3044\u30c7\u30b6\u30a4\u30f3\u3068\u7e4a\u7d30\u306a\u5473\u308f\u3044\u304c\u7279\u5fb4\u3067\u3001\u4e00\u3064\u4e00\u3064\u306e\u548c\u83d3\u5b50\u304c\

In [ ]:
type(result)

openai.openai_object.OpenAIObject

## 4. 呼び出すfunctionと渡すパラメータを確認

In [ ]:
import json

name = result['choices'][0]['message']['function_call']['name']
args = json.loads(result['choices'][0]['message']['function_call']['arguments'])
name, args

('web_site_builder',
 {'title': 'プリティー和菓子コレクション',
  'explanation': 'プリティー和菓子コレクションは、日本の伝統的な和菓子を集めたスイーツのコレクションです。美しいデザインと繊細な味わいが特徴で、一つ一つの和菓子が芸術のような存在です。このコレクションには、桜餅、抹茶の宇治金時、あんこまんじゅうなど、日本で人気のある和菓子が含まれています。桜餅は、もちもちとしたお餅に桜の葉で包まれ、上品な桜の香りと甘さが口の中で広がります。抹茶の宇治金時は、宇治抹茶の風味とあんこの甘さが絶妙にマッチした贅沢な和菓子です。あんこまんじゅうは、もちもちとした生地の中に滑らかなこしあんが包まれており、一つ食べると幸せな気持ちになれます。プリティー和菓子コレクションは、日本の美しい和の世界を味わいたい人におすすめです。是非、一度お試しください！'})

## 5. webで結果を表示

In [ ]:
web_site_builder(**args)